# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [4]:
from autogen import ConversableAgent

writer = ConversableAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [5]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])


>>>>>>>> USING AUTO REPLY...


In [6]:
print(reply)

Title: Unleashing the Power of AI with DeepLearning.AI

Embark on a journey to unlock the realms of Artificial Intelligence with DeepLearning.AI. Founded by the renowned Andrew Ng, this platform offers top-tier courses that delve into the world of deep learning. From convolutional networks to natural language processing, you can master it all. Whether you're a novice or a seasoned expert, DeepLearning.AI caters to all levels, making cutting-edge AI education accessible to everyone. Join the millions worldwide diving into the future of technology. Dive into DeepLearning.AI and equip yourself with the skills to drive innovation and shape tomorrow's world.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [7]:
critic = ConversableAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [8]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Writer (to Critic):

Title: Unleashing the Power of AI with DeepLearning.AI

Embark on a journey to unlock the realms of Artificial Intelligence with DeepLearning.AI. Founded by the renowned Andrew Ng, this platform offers top-tier courses that delve into the world of deep learning. From convolutional networks to natural language processing, you can master it all. Whether you're a novice or a seasoned expert, DeepLearning.AI caters to all levels, making cutting-edge AI education accessible to everyone. Join the millions worldwide diving into the future of technology. Dive into DeepLearning.AI and equip yourself with the skills to drive innovation and shape tomorrow's world.

-----------------------------------------------

## Nested chat

In [11]:
SEO_reviewer = ConversableAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [12]:
legal_reviewer = ConversableAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [13]:
ethics_reviewer = ConversableAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [14]:
meta_reviewer = ConversableAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [15]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [16]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [17]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Writer (to Critic):

Title: Unleashing the Power of AI with DeepLearning.AI

Embark on a journey to unlock the realms of Artificial Intelligence with DeepLearning.AI. Founded by the renowned Andrew Ng, this platform offers top-tier courses that delve into the world of deep learning. From convolutional networks to natural language processing, you can master it all. Whether you're a novice or a seasoned expert, DeepLearning.AI caters to all levels, making cutting-edge AI education accessible to everyone. Join the millions worldwide diving into the future of technology. Dive into DeepLearning.AI and equip yourself with the skills to drive innovation and shape tomorrow's world.

-----------------------------------------------

## Get the summary

In [18]:
print(res.summary)

Title: Master AI with DeepLearning.AI: Courses for Deep Learning Enthusiasts & Beginners

Embark on an AI learning journey with DeepLearning.AI, founded by Andrew Ng. Explore cutting-edge artificial intelligence courses, from convolutional networks to natural language processing. Whether you're new to AI or an expert, our platform caters to all levels. Join millions worldwide shaping the future of technology. Start your AI journey with DeepLearning.AI today and unlock the potential of tomorrow's technology. Transparent, trustworthy, and privacy-focused, we provide top-tier deep learning education. Discover the power of AI and drive innovation with us.
